In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [ ]:
import datasurfer as ds
from datasurfer.lib_llm.utility import agent_tool
from datasurfer.lib_llm.tool import read_file, write_file, web_search, get_current_datetime
from IPython.display import display, Markdown

In [ ]:
@agent_tool
def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the current weather and return a summary.
    location: The city and state, e.g. San Francisco, CA
    unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the current weather
    
    """
    return f"It is currently sunny in {location} and 75 degrees {unit}."

@agent_tool
def get_tomorrows_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the weather for tomorrow and return a summary.
    param location: The city and state, e.g. San Francisco, CA
    param unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the weather for tomorrow
    """
    return f"Tomorrow it will be rainy in {location} and 60 degrees {unit}."

In [ ]:
tools = [get_current_weather, get_tomorrows_weather, read_file, write_file]
mcp_servers = 'http://localhost:8000/mcp'

In [ ]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)

In [ ]:
agent.skills

In [ ]:
skills = agent.skills
_, readref, _ = skills.get_tools()
readref

In [15]:
readref("parse-rss", "\\references\\news_25.rss")

c:\20_Develop\88_Training\71_LLM\skills\parse-rss \references\news_25.rss
c:\references\news_25.rss


"Error executing tool read_skill_reference: name 'path' is not defined"

In [40]:
Markdown(await agent.reset().resp('parse the rss feed'))

Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"parse-rss"}
        Call ID: call_lVSNBgjxiDrTVyQzs4x5EReC


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: read_skill_reference
        Arguments: {"skill_name":"parse-
    rss","reference_link":"\\references\\news_25.rss"}
        Call ID: call_9jmdPArChpOsQsolZsHXZsq6


------------------------------------------------------------------------------------

Assistant:

    I can do that — I tried to follow the parse-rss skill instructions but the
    example/reference file (\references\news_25.rss) could not be found. Please
    provide the RSS feed you want parsed by doing one of the following:  - Paste
    the RSS/XML content here, or   - Send the RSS feed URL, or   - Upload the
    .rss/.xml file.  Also tell me any preferences (optional): - Max number of
    items to return (default: all),   - Includ

I can do that — I tried to follow the parse-rss skill instructions but the example/reference file (\references\news_25.rss) could not be found. Please provide the RSS feed you want parsed by doing one of the following:

- Paste the RSS/XML content here, or  
- Send the RSS feed URL, or  
- Upload the .rss/.xml file.

Also tell me any preferences (optional):
- Max number of items to return (default: all),  
- Include description/summary for each item? (default: no),  
- Convert publication dates to ISO format? (default: yes),  
- Any other fields you want (author, categories, GUID)?

If you prefer the default behavior, send the feed URL or content and I’ll extract titles, links, and publication dates and return a markdown table.

In [10]:
await agent.resp('hi');

Assistant:

    Hello! How can I assist you today?

------------------------------------------------------------------------------------



In [27]:
Markdown(await agent.reset().resp('What is the weather like in Stuttgart Germany today and tomorrow?'))

Assistant:

    Function Call:
        Name: get_current_weather
        Arguments: {"location":"Stuttgart, Germany","unit":"celsius"}
        Call ID: call_NAwnXmGkgUNGFN55FfMEG36E


    Function Call:
        Name: get_tomorrows_weather
        Arguments: {"location":"Stuttgart, Germany","unit":"celsius"}
        Call ID: call_Dw8o3Vxkk26V2WE18QgCEQ8r


------------------------------------------------------------------------------------

Assistant:

    I checked the weather for Stuttgart, but the source returned temperatures
    labeled in Celsius that appear to actually be Fahrenheit. Interpreting them
    as Fahrenheit gives reasonable values. Summary:  - Today (Stuttgart): Sunny
    — about 75°F (≈ 24°C).   - Tomorrow (Stuttgart): Rainy — about 60°F (≈
    16°C).  If you’d like, I can re-check with a different source, give an
    hourly forecast, or show the forecast in only Celsius or only Fahrenheit.
    Which would you prefer?

-------------------------------------------------

I checked the weather for Stuttgart, but the source returned temperatures labeled in Celsius that appear to actually be Fahrenheit. Interpreting them as Fahrenheit gives reasonable values. Summary:

- Today (Stuttgart): Sunny — about 75°F (≈ 24°C).  
- Tomorrow (Stuttgart): Rainy — about 60°F (≈ 16°C).

If you’d like, I can re-check with a different source, give an hourly forecast, or show the forecast in only Celsius or only Fahrenheit. Which would you prefer?

In [15]:
Markdown(await agent.reset().resp('What is the current date and time?'))

Assistant:

    Function Call:
        Name: get_current_time
        Arguments: {}
        Call ID: call_0UBStBJS04Ah28g6LlvxeMVQ


------------------------------------------------------------------------------------

Assistant:

    The current date and time is January 10, 2026, 18:07 (6:07 PM).

------------------------------------------------------------------------------------



The current date and time is January 10, 2026, 18:07 (6:07 PM).

In [26]:
Markdown(await agent.reset().resp('Web search news from Stuttgart of the current month. Return at least 3 news. ' 
' write the results to a file called "stuttgart_news.md"'))   

Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"web-search"}
        Call ID: call_x5zjjimpGeLwOS47iWdpYw2g


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","script_link":"scripts/web_search.py"}
        Call ID: call_hA3DS7OxF6ojE7SMcY4ff0FL


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-search","script_link":"scripts/web_search.
    py","parameters":{"query":"Stuttgart news January
    2026","category":"news","max_results":10}}
        Call ID: call_lUfzaKxzCgMBotgMwBgJRJiF


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Ar

Done. I searched for current-month (January 2026) news about Stuttgart, collected at least three items, and wrote them to stuttgart_news.md. The file includes sources and brief summaries.